In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# Gross
import sys
sys.path.append("..")

In [3]:
import random
from world_rando import generate
from rom_tools.rom_manager import RomManager, region_map_locs
from rom_tools import rom_data_structures
from world_rando.room_dtypes import convert_rooms
from rom_tools.address import Address
from encoding import free_space
from world_rando import map_viz

In [4]:
from rom_tools import rom_edit

In [5]:
rom_m = RomManager("../../roms/sm_clean.smc", "../../roms/sm_infinite_gauntlet1.smc")

In [6]:
a = Address(0x1a03d1)

In [8]:
rom_m.read_from_clean(a, 10)

b'\xbf\xe6\x07\x00\x7f\xea\x07\x00\xff\xff'

In [121]:
print("Parsing original rooms")
parsed_rooms = rom_m.parse()

Parsing original rooms


In [122]:
front_of_gauntlet = parsed_rooms["room_header_0x792b3"]
back_of_gauntlet = parsed_rooms["room_header_0x7965b"]
kraid_save = parsed_rooms["room_header_0x7a70b"]

In [123]:
# Here's the plan
# Create a new right save point
# Put it at the ship save so the game starts there
# Create a bunch of ASP gauntlets
# Hook them together
# ????
# Profit
# [] [] [] [] [] [] [] [] [] [] [] [] [S]

In [124]:
#from world_rando.room_dtypes import Room, Door, RoomCopyConverter, convert_room
import world_rando.room_dtypes as roomd
from world_rando.coord import Coord
from world_rando.settings import Region

In [125]:
gauntlet_str = "gauntlet_{}"
gauntlet_door_str = "{}_to_{}"
crateria = Region("Crateria")
save_name = "save"
back_of_gauntlet_size = 6

In [126]:
def mk_room_copy(room_str, pos, size, rooms, converter):
    room = roomd.Room(None, size, room_str, pos, crateria)
    room.converter = converter
    rooms[room_str] = room
    return room

In [127]:
from collections import namedtuple
GauntletInfo = namedtuple("GauntletInfo", ["converter", "size", "original", "similarity"])

In [128]:
fg = "front_of_gauntlet"
bg = "back_of_gauntlet"

In [129]:
# 0.03 too low for back of gauntlet.
# 0.04 works, but want some more chaos?
# 0.035 is a LITTLE too little
gauntlet_info = {
    fg: GauntletInfo(roomd.RoomCopyConverter("room_header_0x792b3"), Coord(5, 1), front_of_gauntlet, 0.02),
    bg: GauntletInfo(roomd.RoomCopyConverter("room_header_0x7965b"), Coord(6, 1), back_of_gauntlet, 0.036)
}

In [130]:
# Create rooms

In [131]:
rooms = {}

In [132]:
# Save Room
save_pos = Coord(63, 15)
door_s_to_g = roomd.Door(save_pos, Coord(-1, 0), save_name, gauntlet_str.format(0), "save_to_g0")

In [133]:
#TODO: CMAP
save_room = roomd.Room(None, Coord(1,1), save_name, save_pos, crateria)
save_room.doors.append(door_s_to_g)
save_room.converter = roomd.RoomCopyConverter("room_header_0x7a70b")
rooms[save_name] = save_room

In [134]:
#TODO: For now, just add copies of front of gauntlet

In [135]:
door_id = 0
#gauntlets = ["front_of_gauntlet"] * 5
gauntlets = [fg, bg, fg, fg, bg, fg, bg, bg, fg, bg]
pos = save_pos.copy()
for i,g in enumerate(gauntlets):
    ginfo = gauntlet_info[g]
    room_str = gauntlet_str.format(i)
    pos -= Coord(ginfo.size.x, 0)
    print(pos)
    room = mk_room_copy(room_str, pos, ginfo.size, rooms, ginfo.converter)
    # First door is left door
    if i == 0:
        out_name = save_name
    else:
        out_name = gauntlet_str.format(i-1)
    # Traveling to the right, i.e. backwards
    first_door = roomd.Door(pos + Coord(ginfo.size.x - 1, 0), Coord(1, 0), room_str, out_name, f"door_{gauntlet_str}_{door_id}")
    door_id += 1
    room.doors.append(first_door)
    # Traveling to the left, i.e. forwards
    if i+1 != len(gauntlets):
        out_name = gauntlet_str.format(i+1)
    else:
        out_name = gauntlet_str.format(0) # Loop
    second_door = roomd.Door(pos, Coord(-1, 0), room_str, out_name, f"door_{gauntlet_str}_{door_id}")
    door_id += 1
    room.doors.append(second_door)


c(58,15)
c(52,15)
c(47,15)
c(42,15)
c(36,15)
c(31,15)
c(25,15)
c(19,15)
c(14,15)
c(8,15)


In [136]:
#[53, 54, 55, 56, 57,] [58, 59, 60, 61, 62,] [63,]

In [137]:
# End create rooms

In [138]:
rooms
[[d.destination for d in r.doors] for r in rooms.values()]

[['gauntlet_0'],
 ['save', 'gauntlet_1'],
 ['gauntlet_0', 'gauntlet_2'],
 ['gauntlet_1', 'gauntlet_3'],
 ['gauntlet_2', 'gauntlet_4'],
 ['gauntlet_3', 'gauntlet_5'],
 ['gauntlet_4', 'gauntlet_6'],
 ['gauntlet_5', 'gauntlet_7'],
 ['gauntlet_6', 'gauntlet_8'],
 ['gauntlet_7', 'gauntlet_9'],
 ['gauntlet_8', 'gauntlet_0']]

In [139]:
new_obj_names = convert_rooms(rooms, parsed_rooms)

In [140]:
[d for d in new_obj_names.values() if type(d) is rom_data_structures.RoomHeader]

[room_header_id_0save,
 room_header_id_0gauntlet_0,
 room_header_id_0gauntlet_1,
 room_header_id_0gauntlet_2,
 room_header_id_0gauntlet_3,
 room_header_id_0gauntlet_4,
 room_header_id_0gauntlet_5,
 room_header_id_0gauntlet_6,
 room_header_id_0gauntlet_7,
 room_header_id_0gauntlet_8,
 room_header_id_0gauntlet_9]

In [141]:
header_str = "room_header_id_0gauntlet_{}"

In [142]:
new_obj_names[header_str.format(len(gauntlets)-1)].door_list.l[1].list

[room_header_id_0gauntlet_0, 0, 5, -802, 6, -51, 0, 32768, None]

In [143]:
# Special case: Edit the door
new_obj_names[header_str.format(len(gauntlets)-1)].door_list.l[1].x_pos_low = 78
new_obj_names[header_str.format(len(gauntlets)-1)].door_list.l[1].x_pos_high = gauntlet_info[gauntlets[0]].size.x - 1

In [144]:
[d.list for d in new_obj_names.values() if type(d) is rom_data_structures.Door]

[[room_header_id_0gauntlet_0, 0, 5, 78, 6, 4, 0, 32768, None],
 [room_header_id_0save, 0, 4, 1, 6, 0, 0, 32768, None],
 [room_header_id_0gauntlet_1, 0, 5, 94, 6, 5, 0, 32768, None],
 [room_header_id_0gauntlet_0, 0, 4, 1, 6, 0, 0, 32768, None],
 [room_header_id_0gauntlet_2, 0, 5, 78, 6, 4, 0, 32768, None],
 [room_header_id_0gauntlet_1, 0, 4, 1, 6, 0, 0, 32768, None],
 [room_header_id_0gauntlet_3, 0, 5, 78, 6, 4, 0, 32768, None],
 [room_header_id_0gauntlet_2, 0, 4, 1, 6, 0, 0, 32768, None],
 [room_header_id_0gauntlet_4, 0, 5, 94, 6, 5, 0, 32768, None],
 [room_header_id_0gauntlet_3, 0, 4, 1, 6, 0, 0, 32768, None],
 [room_header_id_0gauntlet_5, 0, 5, 78, 6, 4, 0, 32768, None],
 [room_header_id_0gauntlet_4, 0, 4, 1, 6, 0, 0, 32768, None],
 [room_header_id_0gauntlet_6, 0, 5, 94, 6, 5, 0, 32768, None],
 [room_header_id_0gauntlet_5, 0, 4, 1, 6, 0, 0, 32768, None],
 [room_header_id_0gauntlet_7, 0, 5, 94, 6, 5, 0, 32768, None],
 [room_header_id_0gauntlet_6, 0, 4, 1, 6, 0, 0, 32768, None],
 [room

In [145]:
# Add the savestation
save = new_obj_names["room_header_id_0save"]
room_outside_save = save.door_list.l[0].to_room
door_to_save = [d for d in room_outside_save.door_list.l if d.to_room is save][0]
#['room', 'door', 'door_bts', 'save_x', 'save_y', 'samus_x', 'samus_y']
start_replace = [save, door_to_save, 0, 0, 0, 152, 0]
new_obj_names.copy_obj(parsed_rooms["save_station_0x44c5"], start_replace)

save_station_0x44c5

In [146]:
#new_obj_names.d

In [147]:
len(new_obj_names)

115

In [148]:
#rom_m.memory.banks
rom_m.clear_memory()

In [149]:
rom_edit.make_starting_items("B MB M5 E499 GS SA SPB Spazer", rom_m)

In [150]:
#TODO: cmaps
#TODO: alter the PLM lists + change the PLM IDs

In [151]:
sys.path.append("../../wfc_2019f")
from world_rando import asp_wave_collapse
from tqdm.notebook import tqdm

In [152]:
import random

In [153]:
# Alter the level data
#TODO: cache the patterns
#TODO: sanity check that each door is in the same BFS component
#TODO: Allow accessing enemies as a new layer, then randomize enemy locations as well
# (still with some % fixed)
#TODO: Save station reload hangs the game?
#TODO: Door ASM for fixing scrolls when entering from the left for back_of_gauntlet
for i, g in enumerate(tqdm(gauntlets)):
    i_header = new_obj_names[header_str.format(i)]
    original = gauntlet_info[g].original
    # Get the level data using the original
    sim = gauntlet_info[g].similarity
    level, fns = asp_wave_collapse.wfc_level_data(original, seed=random.random(), extra_similarity=sim)
    #print(fns)
    # Save the level data to the new level
    asp_wave_collapse.create(i_header, level, fns, keep_allocation=False)

  0%|          | 0/10 [00:00<?, ?it/s]

room_header_0x792b3
For rectangle (c(0,0),c(80,16)):
Fixed tile ratio: 0.190625
	Getting tile grid
	Rect size: (80, 16)
	Getting pattern catalog
	Number of patterns: 329
	Getting adjacencies
	Starting ASP instance
	Grounding ASP Model
	Solving ASP Model
room_header_0x7965b
For rectangle (c(0,0),c(96,16)):
Fixed tile ratio: 0.18619791666666666
	Getting tile grid
	Rect size: (96, 16)
	Getting pattern catalog
	Number of patterns: 432
	Getting adjacencies
	Starting ASP instance
	Grounding ASP Model
	Solving ASP Model
room_header_0x792b3
For rectangle (c(0,0),c(80,16)):
Fixed tile ratio: 0.190625
	Getting tile grid
	Rect size: (80, 16)
	Getting pattern catalog
	Number of patterns: 329
	Getting adjacencies
	Starting ASP instance
	Grounding ASP Model
	Solving ASP Model
room_header_0x792b3
For rectangle (c(0,0),c(80,16)):
Fixed tile ratio: 0.190625
	Getting tile grid
	Rect size: (80, 16)
	Getting pattern catalog
	Number of patterns: 329
	Getting adjacencies
	Starting ASP instance
	Grounding AS

In [154]:
c = new_obj_names["room_header_id_0gauntlet_0"].state_chooser.conditions[0]
c = front_of_gauntlet.state_chooser.conditions[0]

In [155]:
c.state.level_data, front_of_gauntlet.state_chooser.default.level_data

(level_data_0x2156e8, level_data_0x2156e8)

In [156]:
# Reset new_obj_names
#for d in new_obj_names.values():
#    if hasattr(d, "address"):
#        delattr(d, "address")

In [157]:
print("Compiling rooms")
rom_m.compile(new_obj_names)
rom_m.save_and_close()

Compiling rooms
Saves: [save_station_0x44c5]
ALLOCATION
Allocating compiled object door_6 of size 12 at 0x1ad70 in 0x83: [($0x1ad7c, 0x5284)]. Old size: None
Allocating compiled object door_23 of size 12 at 0x1ad7c in 0x83: [($0x1ad88, 0x5278)]. Old size: None
Allocating compiled object door_35 of size 12 at 0x1ad88 in 0x83: [($0x1ad94, 0x526c)]. Old size: None
Allocating compiled object door_42 of size 12 at 0x1ad94 in 0x83: [($0x1ada0, 0x5260)]. Old size: None
Allocating compiled object door_49 of size 12 at 0x1ada0 in 0x83: [($0x1adac, 0x5254)]. Old size: None
Allocating compiled object door_54 of size 12 at 0x1adac in 0x83: [($0x1adb8, 0x5248)]. Old size: None
Allocating compiled object door_61 of size 12 at 0x1adb8 in 0x83: [($0x1adc4, 0x523c)]. Old size: None
Allocating compiled object door_66 of size 12 at 0x1adc4 in 0x83: [($0x1add0, 0x5230)]. Old size: None
Allocating compiled object door_71 of size 12 at 0x1add0 in 0x83: [($0x1addc, 0x5224)]. Old size: None
Allocating compile

In [403]:
door_l = new_obj_names["room_header_id_0gauntlet_0"].door_list.l

In [272]:
door_l

[door_6, door_7]

In [267]:
[d.to_room for d in door_l]
[d.list for d in door_l]

[[room_header_id_0save, 0, 4, 1, 6, 0, 0, 32768, None],
 [room_header_id_0gauntlet_1, 0, 5, 78, 6, 4, 0, 32768, None]]

In [268]:
door_l_2 = new_obj_names["room_header_id_0save"].door_list.l

In [269]:
door_l_2[0].to_room

room_header_id_0gauntlet_0

In [270]:
parsed_rooms["save_station_0x45f9"].list

[room_header_0x7a70b, door_0x19186, 0, 0, 0, 152, 0]

In [271]:
[d.to_room for d in front_of_gauntlet.door_list.l]
[d.list for d in front_of_gauntlet.door_list.l]

[[room_header_0x791f8, 0, 4, 1, 38, 0, 2, 32768, placeholder_0x7b997],
 [room_header_0x7965b, 0, 5, 94, 6, 5, 0, 32768, None]]

In [275]:
rom_m2 = RomManager("../../roms/sm_infinite_gauntlet.smc", "../../roms/sm_whatever.smc")

In [278]:
from rom_tools.rom_data_structures import parse_from_savestations

In [293]:
hex(Address(0x1addc).as_snes)

'0x83addc'

In [302]:
[(o, type(o).__name__, o.address) for o in new_obj_names.values() if hasattr(o, "address")]

[(door_1, 'Door', 0x1addc),
 (door_list_2, 'DoorList', 0x7e9ad),
 (room_header_id_0save, 'RoomHeader', 0x7a70b),
 (level_data_0x2715c2, 'LevelData', 0x2715c2),
 (fx_0x1842c, 'FX', 0x1842c),
 (enemy_list_0x1093a9, 'EnemyList', 0x1093a9),
 (enemy_types_0x1a03c7, 'EnemyTypes', 0x1a03c7),
 (PLM_list_0x78ad2, 'PLMList', 0x78ad2),
 (placeholder_0x791f4, 'Placeholder', 0x791f4),
 (door_6, 'Door', 0x1ad70),
 (door_7, 'Door', 0x1add0),
 (door_list_8, 'DoorList', 0x7e9a9),
 (room_header_id_0gauntlet_0, 'RoomHeader', 0x792b3),
 (room_state_12, 'RoomState', 0x792df),
 (level_data_0x2156e8, 'LevelData', 0x2156e8),
 (fx_0x180e0, 'FX', 0x180e0),
 (enemy_list_0x10847a, 'EnemyList', 0x10847a),
 (enemy_types_0x1a00a1, 'EnemyTypes', 0x1a00a1),
 (PLM_list_0x78058, 'PLMList', 0x78058),
 (placeholder_0x7b899, 'Placeholder', 0x7b899),
 (placeholder_0x791d3, 'Placeholder', 0x791d3),
 (door_23, 'Door', 0x1ad7c),
 (door_24, 'Door', 0x1adc4),
 (door_list_25, 'DoorList', 0x7e9a5),
 (room_header_id_0gauntlet_1, 'R

In [301]:
print("Parsing original rooms")
inf_rooms = parse_from_savestations([Address(0x44c5)], rom_m2)

Parsing original rooms
Parsing: save_station_0x44c5
Using pointer_parser at 0x44c5
Pointer to 0x7a70b
Parsing: room_header_0x7a70b
Using parse_int at 0x7a70b
Got: 54, Size: 1 using parse_int at 0x7a70b

Using enum_parser at 0x7a70c
Got: 0, Size: 1 using enum_parser at 0x7a70c

Using parse_int at 0x7a70d
Got: 63, Size: 1 using parse_int at 0x7a70d

Using parse_int at 0x7a70e
Got: 15, Size: 1 using parse_int at 0x7a70e

Using parse_int at 0x7a70f
Got: 1, Size: 1 using parse_int at 0x7a70f

Using parse_int at 0x7a710
Got: 1, Size: 1 using parse_int at 0x7a710

Using parse_int at 0x7a711
Got: 112, Size: 1 using parse_int at 0x7a711

Using parse_int at 0x7a712
Got: 160, Size: 1 using parse_int at 0x7a712

Using enum_parser at 0x7a713
Got: set(), Size: 1 using enum_parser at 0x7a713

Using pointer_parser at 0x7a714
Pointer to 0x7e9ad
Parsing: door_list_0x7e9ad
Using list_parser at 0x7e9ad
Pointer to 0x1addc
Parsing: door_0x1addc
Using pointer_parser at 0x1addc
Pointer to 0x792b3
Parsing: roo

AssertionError: No matching enum entry for 193

In [289]:
f"{Address.__name__}"

'Address'